<h1 style="font-size:30px;">Application: Lane Detection in video</h1> 

This notebook will help you to detect lanes in a video using OpenCV. In the previous notebook, you learned how to detect lanes in a single image. In this notebook, you will see how to apply this pipeline to a video example, as well as consider of some of the limitations and possible extensions.


In [1]:
import numpy as np
import cv2

if 'google.colab' in str(get_ipython()):
    print("Downloading Code to Colab Environment")
    !wget https://www.dropbox.com/sh/93k0ei59i6p554o/AAD1WAU_u25zZCs5LqyBDhaZa?dl=1 -O module-code.zip -q --show-progress
    !unzip -qq module-code.zip
else:
    pass

In [9]:
from moviepy.editor import *
# from moviepy.video.io.html_tools import ipython_display
clip = VideoFileClip('lane1-straight.mp4')
clip.set_fps(30)
clip.ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



TypeError: must be real number, not NoneType

# 1. Define Utility Functions From Straight Lane Image

Below are several utility functions that we will use to run our pipeline.

### <font style='color:rgb(50,120,230)'> 1.1 Function from code ocvered in straight lane in image detection

In [ ]:
def region_of_interest(img, vertices):
    """Select the region of interest (ROI) from a defined list of vertices."""
    # Defines a blank mask.
    mask = np.zeros_like(img)   
    
    # Defining a 3 channel or 1 channel color to fill the mask.
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # 3 or 4 depending on your image.
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Filling pixels inside the polygon.
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Returning the image only where mask pixels are nonzero.
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness = 2):
    """Utility for drawing lines."""
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """Utility for defining Line Segments."""
    lines = cv2.HoughLinesP(
        img, rho, theta, threshold, np.array([]),
        minLineLength = min_line_len, maxLineGap = max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines


def separate_left_right_lines(lines):
    """Separate left and right lines depending on the slope."""
    left_lines = []
    right_lines = []
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if y1 > y2: # Negative slope = left lane.
                    left_lines.append([x1, y1, x2, y2])
                elif y1 < y2: # Positive slope = right lane.
                    right_lines.append([x1, y1, x2, y2])
    return left_lines, right_lines


def cal_avg(values):
    """Calculate average value."""
    if not (type(values) == 'NoneType'):
        if len(values) > 0:
            n = len(values)
        else:
            n = 1
        return sum(values) / n


def extrapolate_lines(lines, upper_border, lower_border):
    """Extrapolate lines keeping in mind the lower and upper border intersections."""
    slopes = []
    consts = []
    
    if (lines is not None) and (len(lines) != 0):
        for x1, y1, x2, y2 in lines:
            slope = (y1-y2) / (x1-x2)
            slopes.append(slope)
            c = y1 - slope * x1
            consts.append(c)
        avg_slope = cal_avg(slopes)
        avg_consts = cal_avg(consts)

        # Calculate average intersection at lower_border.
        x_lane_lower_point = int((lower_border - avg_consts) / avg_slope)

        # Calculate average intersection at upper_border.
        x_lane_upper_point = int((upper_border - avg_consts) / avg_slope)

        return [x_lane_lower_point, lower_border, x_lane_upper_point, upper_border]


def extrapolated_lane_image(img, lines, roi_upper_border, roi_lower_border):
    """Main function called to get the final lane lines."""
    lanes_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    # Extract each lane.
    lines_left, lines_right = separate_left_right_lines(lines)
    lane_left = extrapolate_lines(lines_left, roi_upper_border, roi_lower_border)
    lane_right = extrapolate_lines(lines_right, roi_upper_border, roi_lower_border)
    if lane_left is not None and lane_right is not None:
        draw_con(lanes_img, [[lane_left], [lane_right]])
    return lanes_img


### <font style='color:rgb(50,120,230)'> 1.2 New function for drawing area between the lanes.


In [ ]:
def draw_con(img, lines):
    """Fill in lane area."""
    points = []
    for x1,y1,x2,y2 in lines[0]:
        points.append([x1,y1])
        points.append([x2,y2])
    for x1,y1,x2,y2 in lines[1]:
        points.append([x2,y2])
        points.append([x1,y1])

    points = np.array([points], dtype = 'int32')        
    cv2.fillPoly(img, points, (0,255,0))

# 2. Create the Main Process Loop Function

This will be called over each frame of our dashcam video feed.

In [ ]:
def process_image(image):  
    # Convert to grayscale.
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Intensity selection.
    gray_select = cv2.inRange(gray, 150, 255)
    
    # Region masking: Select vertices according to the input image.
    roi_vertices = np.array([[[100, 540], [900, 540], [525, 330], [440, 330]]])
    gray_select_roi = region_of_interest(gray_select, roi_vertices)
    
    # Canny Edge Detection.
    low_threshold = 50
    high_threshold = 100
    img_canny = cv2.Canny(gray_select_roi, low_threshold, high_threshold)
    
    # Remove noise using Gaussian blur.
    kernel_size = 5
    canny_blur = cv2.GaussianBlur(img_canny, (kernel_size, kernel_size), 0)
    
    # Hough transform parameters set according to the input image.
    rho = 1
    theta = np.pi/180
    threshold = 100
    min_line_len = 50
    max_line_gap = 300
    hough, lines = hough_lines(canny_blur, rho, theta, threshold, min_line_len, max_line_gap)
    
    # Extrapolate lanes.
    roi_upper_border = 330
    roi_lower_border = 540
    lane_img = extrapolated_lane_image(image, lines, roi_upper_border, roi_lower_border)
    
    # Combined using weighted image.
    image_result = cv2.addWeighted(image, 1, lane_img, 0.4, 0.0)
    return image_result

# 3. Setup video capture

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font color="green">OpenCV Documentation</font>

[**`VideoCapture()`**](https://docs.opencv.org/4.5.2/d8/dfe/classcv_1_1VideoCapture.html#ac4107fb146a762454a8a87715d9b7c96) 

[**`VideoWriter()`**](https://docs.opencv.org/4.5.2/dd/d9e/classcv_1_1VideoWriter.html#ad59c61d8881ba2b2da22cff5487465b5) 

[**`write()`**](https://docs.opencv.org/4.5.2/dd/d9e/classcv_1_1VideoWriter.html#a30ebbc09c122332f62bd706b43f02a98)

[**`release()`**](https://docs.opencv.org/4.5.2/dd/d9e/classcv_1_1VideoWriter.html#a667f737e56d5ba6b0533c6c7bf941140) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

In [ ]:
# Initialize our video capture.
video_cap = cv2.VideoCapture('lane1-straight.mp4')
if not video_cap.isOpened(): 
  print("Error opening video stream or file")

In [ ]:
# Retrieve video frame properties.
frame_w   = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h   = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_fps = int(video_cap.get(cv2.CAP_PROP_FPS))

# Select fourcc encoding for the mp4 file.
# Having issues? You could also try: *'mp4v' or *'avc1'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Specify the video output filenames.
file_out = 'lane1-straight-output.mp4'

# Create the video writer objects.
vid_out = cv2.VideoWriter(file_out, fourcc, frame_fps, (frame_w,frame_h))

In [ ]:
# Run the main loop over every frame of the input video.
print("Begin processing video... Wait until 'finished' message!")
while True:
    ret, frame = video_cap.read()
    if frame is None:
        print("Finished processing video")
        break
    result = process_image(frame)
    vid_out.write(result)

# Close the video writer stream.
vid_out.release()

In [ ]:
# Display the saved video file.
from moviepy.editor import *
clip = VideoFileClip(file_out)
clip.ipython_display(width = 800)


Now you are able to detect straight lane lines. It is worth understanding some of the hard coded assumption this pipeline took around area of interest. You can consider possible extensions, such as detecting the horizon level and adjusting the size of the trapezoid region of interest accordingly.

Furthermore, in real world scenario, the roads are not always straight. Additionally, sometimes the visibility might be low. You can go a step further and see how this pipeline performs over some extreme situations.